## Raw IMU Data

In [2]:
import os
import json
import numpy as np
import torch

## functions to use
def from_quaternion(quat, ordering='wxyz'):
        """Form a rotation matrix from a unit length quaternion.
        Valid orderings are 'xyzw' and 'wxyz'.
        """
        if ordering == 'xyzw':
            qx = quat[:, 0]
            qy = quat[:, 1]
            qz = quat[:, 2]
            qw = quat[:, 3]
        elif ordering == 'wxyz':
            qw = quat[:, 0]
            qx = quat[:, 1]
            qy = quat[:, 2]
            qz = quat[:, 3]

        # Form the matrix
        mat = quat.new_empty(quat.shape[0], 3, 3)

        qx2 = qx * qx
        qy2 = qy * qy
        qz2 = qz * qz

        mat[:, 0, 0] = 1. - 2. * (qy2 + qz2)
        mat[:, 0, 1] = 2. * (qx * qy - qw * qz)
        mat[:, 0, 2] = 2. * (qw * qy + qx * qz)

        mat[:, 1, 0] = 2. * (qw * qz + qx * qy)
        mat[:, 1, 1] = 1. - 2. * (qx2 + qz2)
        mat[:, 1, 2] = 2. * (qy * qz - qw * qx)

        mat[:, 2, 0] = 2. * (qx * qz - qw * qy)
        mat[:, 2, 1] = 2. * (qw * qx + qy * qz)
        mat[:, 2, 2] = 1. - 2. * (qx2 + qy2)
        return mat

def slerp(q0, q1, tau, DOT_THRESHOLD = 0.9995):
    """Spherical linear interpolation."""

    dot = (q0*q1).sum(dim=1)
    q1[dot < 0] = -q1[dot < 0]
    dot[dot < 0] = -dot[dot < 0]

    q = torch.zeros_like(q0)
    tmp = q0 + tau.unsqueeze(1) * (q1 - q0)
    tmp = tmp[dot > DOT_THRESHOLD]
    q[dot > DOT_THRESHOLD] = tmp / tmp.norm(dim=1, keepdim=True)

    theta_0 = dot.acos()
    sin_theta_0 = theta_0.sin()
    theta = theta_0 * tau
    sin_theta = theta.sin()
    s0 = (theta.cos() - dot * sin_theta / sin_theta_0).unsqueeze(1)
    s1 = (sin_theta / sin_theta_0).unsqueeze(1)
    q[dot < DOT_THRESHOLD] = ((s0 * q0) + (s1 * q1))[dot < DOT_THRESHOLD]
    return q / q.norm(dim=1, keepdim=True)

def qinterp(qs, t, t_int):
    idxs = np.searchsorted(t, t_int)
    idxs0 = idxs-1
    idxs0[idxs0 < 0] = 0
    idxs1 = idxs
    idxs1[idxs1 == t.shape[0]] = t.shape[0] - 1
    q0 = qs[idxs0]
    q1 = qs[idxs1]
    tau = torch.zeros_like(t_int)
    dt = (t[idxs1]-t[idxs0])[idxs0 != idxs1]
    tau[idxs0 != idxs1] = (t_int-t[idxs0])[idxs0 != idxs1]/dt
    return slerp(q0, q1, tau)

def qnorm(q):
    "Quaternion normalization"
    return q / q.norm(dim=1, keepdim=True)


def interpolate(x, t, t_int):
    """
    Interpolate ground truth at the sensor timestamps
    """

    # vector interpolation
    x_int = np.zeros((t_int.shape[0], x.shape[1]))
    for i in range(x.shape[1]):
        if i in [4, 5, 6, 7]:
            continue
        x_int[:, i] = np.interp(t_int, t, x[:, i])
    # quaternion interpolation
    t_int = torch.Tensor(t_int - t[0])
    t = torch.Tensor(t - t[0])
    qs = qnorm(torch.Tensor(x[:, 4:8]))
    x_int[:, 4:8] = qinterp(qs, t, t_int).numpy()
    return x_int

# Get the current directory
current_dir = os.getcwd()

# List all directories in the current directory
folders = [f for f in os.listdir(current_dir) if os.path.isdir(os.path.join(current_dir, f))]

print(folders)

for dir_name in folders:
    print("Processing folder: ", dir_name)
    imu_path = os.path.join(current_dir, dir_name, "mav0/imu0/data.csv")
    gt_path = os.path.join(current_dir, dir_name, "mav0/state_groundtruth_estimate0/data.csv")

    imu = np.genfromtxt(imu_path, delimiter=",", skip_header=1)
    gt = np.genfromtxt(gt_path, delimiter=",", skip_header=1)

    # time synchronization between IMU and ground truth
    t0 = np.max([gt[0, 0], imu[0, 0]])
    t_end = np.min([gt[-1, 0], imu[-1, 0]])

    # start index
    idx0_imu = np.searchsorted(imu[:, 0], t0)
    idx0_gt = np.searchsorted(gt[:, 0], t0)

    # end index
    idx_end_imu = np.searchsorted(imu[:, 0], t_end, 'right')
    idx_end_gt = np.searchsorted(gt[:, 0], t_end, 'right')

    # subsample
    imu = imu[idx0_imu: idx_end_imu]
    gt = gt[idx0_gt: idx_end_gt]
    ts_imu = imu[:, 0]

    # interpolate
    gt = interpolate(gt, gt[:, 0], ts_imu) # quaternion order: w, x, y, z

    ts_data = ts_imu / 1e3 # convert to microseconds

    assert (ts_data[-1] - ts_data[0]) /1e6 > 0 and (ts_data[-1] - ts_data[0])/1e6 < 380.0, "The duration of the dataset is: " + str((ts_data[-1] - ts_data[0])/1e6)
    gyr_data = imu[:, 1:4]
    acc_data = imu[:, 4:7]
    q_data = gt[:, 4:8] # order: w, x, y, z
    R_rot = from_quaternion(torch.from_numpy(q_data), ordering='wxyz')
    # convert to order: x, y, z, w
    q_data = q_data[:, [1, 2, 3, 0]]
    pos_data = gt[:, 1:4]
    vel_data = gt[:, 8:11]
    # convert vel_world to vel_body
    # R_rot = from_quaternion(torch.from_numpy(q_data), ordering='xyzw')
    vel_data = (R_rot.transpose(-1,-2) @ torch.from_numpy(vel_data).unsqueeze(-1)).squeeze(-1).numpy()
    # print("vel_data[0]", vel_data[0])

    # Write npy files
    data_save = np.hstack((ts_data.reshape(-1, 1), gyr_data, acc_data, q_data, pos_data, vel_data))
    np.save(dir_name + '/imu0_resampled.npy', data_save)

    # Write JSON file
    num_rows = imu.shape[0]
    t_start_us = imu[0, 0]
    t_end_us = imu[-1, 0]
    json_data = {
    "columns_name(width)": [
        "ts_us(1)",
        "gyr_compensated_rotated_in_World(3)",
        "acc_compensated_rotated_in_World(3)",
        "qxyzw_World_Device(4)",
        "pos_World_Device(3)",
        "vel_Body(3)"
    ],
    "num_rows": num_rows,
    "approximate_frequency_hz": 200.0,
    "t_start_us": t_start_us,
    "t_end_us": t_end_us
    }    
    with open(dir_name+ '/imu0_resampled_description.json', 'w') as json_file:
        json.dump(json_data, json_file, indent=4)  # 'indent=4' makes the JSON file readable (pretty-printed)

    



['zhicheng_body1', 'xiaojing_body2', 'hang_body_test1', 'ruixuan_body1', 'hang_body_backward2', 'hang_body_slow1', 'ma_body2', 'hang_body_normal1', 'hang_body_backward1', 'zhicheng_body2', 'dan_body2', 'ma_body1', 'yajie_body2', 'hang_body_side1', 'hang_body_fast1', 'dan_body3', 'hang_body_stop1', 'xiaojing_body1', 'tang_body1', 'huayi_body_test1', 'ruixuan_body2', 'hao_body1', 'tang_body2', 'hang_body_backward4', 'hang_body_backward3', 'dan_body1', 'yajie_body1', 'hao_body2']
Processing folder:  zhicheng_body1
Processing folder:  xiaojing_body2
Processing folder:  hang_body_test1
Processing folder:  ruixuan_body1
Processing folder:  hang_body_backward2
Processing folder:  hang_body_slow1
Processing folder:  ma_body2
Processing folder:  hang_body_normal1
Processing folder:  hang_body_backward1
Processing folder:  zhicheng_body2
Processing folder:  dan_body2
Processing folder:  ma_body1
Processing folder:  yajie_body2
Processing folder:  hang_body_side1
Processing folder:  hang_body_fas

## Select Train, Val and Test Data

In [2]:
import os
import random
import math

current_dir = os.getcwd()
print("current_dir)", current_dir)
seq = [f for f in os.listdir(current_dir) if os.path.isdir(os.path.join(current_dir, f))]
random.shuffle(seq)

num_val = math.floor(len(seq) // 10)
num_test = math.floor(len(seq) // 10)
index_end_train = len(seq) - num_val - num_test
print("total number of sequences: ", len(seq))
print("number of training sequences: ", index_end_train)
print("number of validation sequences: ", num_val)
print("number of test sequences: ", num_test)

all_ids_txt_path = os.path.join(current_dir, 'all_ids.txt')
train_list_txt_path = os.path.join(current_dir, 'train_list.txt')
val_list_txt_path = os.path.join(current_dir, 'val_list.txt')
test_list_txt_path = os.path.join(current_dir, 'test_list.txt')
with open(all_ids_txt_path, 'w') as f:
    pass
with open(train_list_txt_path, 'w') as f:
    pass
with open(val_list_txt_path, 'w') as f:
    pass
with open(test_list_txt_path, 'w') as f:
    pass
for i, seq in enumerate(seq):
    with open(all_ids_txt_path, 'a') as f:
        f.write(seq + '\n')
    if i < index_end_train:
        with open(os.path.join(current_dir, 'train_list.txt'), 'a') as f:
            f.write(seq + '\n')
    elif i < index_end_train + num_val:
        with open(os.path.join(current_dir, 'val_list.txt'), 'a') as f:
            f.write(seq + '\n')
    else:
        with open(os.path.join(current_dir, 'test_list.txt'), 'a') as f:
            f.write(seq + '\n')

current_dir) /home/sangli/Ben/code/imu-equivariant-learning/local_data_bodyframe/RIDI
total number of sequences:  28
number of training sequences:  24
number of validation sequences:  2
number of test sequences:  2


In [3]:
import os

ridi_data_path = '/home/sangli/Downloads/DATASET/ridi-robust-imu-double-integration/versions/1/data_publish_v2/dan_bag1'
path_tmp = os.path.join(ridi_data_path, 'processed/')

current_dir) /home/sangli/Ben/code/imu-equivariant-learning/local_data_bodyframe/RIDI
